In [74]:
import argparse
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy import sparse
from dataset import *
from dataloader import *
from loss import *
from model import *
from torch.nn import Softmax
import os
import bottleneck as bn
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [75]:
if torch.cuda.is_available():
        cuda_aval = True

device = torch.device("cuda" if cuda_aval else "cpu")
print("Now using device : ", device)
###################
print("Load and Preprocess Movielens dataset")
# Load Data
uim = pd.read_csv("new_user_movie_interaction.csv")
# uim = pd.read_csv("/opt/ml/input/workspace/CF/DL CF/Mult-VAE/new_user_movie_interaction.csv")
user_id = uim["user"].to_numpy()
uim_np = uim.to_numpy()[:, 1:]
print("Done!")

Now using device :  cuda
Load and Preprocess Movielens dataset
Done!


In [76]:
DATA_DIR='/opt/ml/input/data/train/'
pro_dir = os.path.join(DATA_DIR, 'pro_sg')

unique_sid = list()
with open(os.path.join(pro_dir, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

show2id = dict([(int(sid), i) for (i, sid) in enumerate(unique_sid)]) # {4643: 0,170: 1,531: 2,616: 3,2140: 4,
item_id = np.array(unique_sid, dtype=int).reshape(len(unique_sid)) #  4643,    170,    531

In [77]:
#load model
f = open("model.pt", 'rb')
model  = torch.load(f).to(device)

In [78]:
model.eval()
result = list()
for i in range(len(user_id)) : 
    user = user_id[i]

    data_np = uim_np[i]
    data = torch.FloatTensor(data_np)
    data = data.reshape(1,-1).to(device)

    probability, _, _ = model(data)
    probability_np = np.array(probability.cpu().detach())[0]
    probability_np[data_np.nonzero()] = -np.inf
    idx = np.argpartition(probability_np, -10)[-10:] # 상위 10개 뽑음 (정렬 x)
    
    candidates = [[index, probability_np[index]] for index in idx]
    candidates.sort(key=lambda x:x[1], reverse = True) # 상위 10개 정렬
    items = item_id[idx]
    for candidate in candidates :
        result.append((user, item_id[candidate[0]]))
    if i%5000 == 0 :
        print(f"{i}/31360 complete")

0/31360 complete


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


5000/31360 complete
10000/31360 complete
15000/31360 complete
20000/31360 complete
25000/31360 complete
30000/31360 complete


In [79]:
info = pd.DataFrame(result, columns=['user','item'])
info.to_csv("submission_earlystop.csv",index=False)
print("Inference Done!")

Inference Done!
